In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os.path
import acquire
import re

%matplotlib inline

In [ ]:
url_data_dict_311 = 'https://data.sanantonio.gov/dataset/93b0e7ee-3a55-4aa9-b27b-d1817e91aec3/resource/0eab0cfc-a114-4064-9f25-d088d8ac46a3/download/datadictionary-311.xlsx'
data_dictionary_311_new = pd.read_excel(url_data_dict_311, header=2)
data_dictionary_311_new

In [12]:
df_311 = acquire.acquire_311()
df_311.head(3)

,Category,CASEID,OPENEDDATETIME,SLA_Date,CLOSEDDATETIME,Late (Yes/No),Dept,REASONNAME,TYPENAME,CaseStatus,SourceID,OBJECTDESC,Council District,XCOORD,YCOORD,Report Starting Date,Report Ending Date
0,Streets & Infrastructure,1010126853,12/10/2011,12/11/2011,3/5/2020,YES,Trans & Cap Improvements,Storm Water,Tree Fallen,Closed,Constituent Call,ELM CREEK and WURZBACH RD,8,2113584.0,13747492.0,6/13/2019,6/13/2020
1,Streets & Infrastructure,1010126880,12/10/2011,12/13/2011,3/5/2020,YES,Trans & Cap Improvements,Streets,Barricade - Misc. Special Event/Emergency,Closed,Constituent Call,"1600 CENTENNIAL BV, San Antonio, 78211",5,2116553.0,13684805.0,6/13/2019,6/13/2020
2,Information,1010338011,5/25/2012,NaN,1/27/2020,NO,Knowledge Management,Knowledge,Knowledge Solution Extra,Closed,Constituent Call,"310 ROOSEVELT AV, San Antonio, 78210",5,2132295.0,13693603.0,6/13/2019,6/13/2020


In [ ]:
df_weather = acquire.acquire_weather()

In [3]:
df_sso = acquire.acquire_sso()

## Prepare

In [ ]:
df_311.Category.value_counts()

In [ ]:
df_311.info()

In [ ]:
df_311.shape

In [ ]:
df_311.Dept.isna().sum() / df_311.shape[0]

In [ ]:
df_311.SLA_Date.isna().sum() / df_311.shape[0]

In [ ]:
df_311.CLOSEDDATETIME.isna().sum() / df_311.shape[0]

In [10]:
df_311 = df_311.rename(columns=
        {
            "Late (Yes/No)": "Late", 
            "Council District": "Council_District", 
            "Report Starting Date": "Report_Start_Date", 
            "Report Ending Date": "Report_Ending_Date"
        })

In [11]:
df_311_cols = df_311.columns.to_list()
df_311_cols = [c.lower() for c in df_311_cols]

In [6]:
df_311_cols

['category',
 'caseid',
 'openeddatetime',
 'sla_date',
 'closeddatetime',
 'late',
 'dept',
 'reasonname',
 'typename',
 'casestatus',
 'sourceid',
 'objectdesc',
 'council_district',
 'xcoord',
 'ycoord',
 'report_start_date',
 'report_ending_date']

In [ ]:
df_311.head(3)

In [ ]:
df_sso.head(3)

In [ ]:
df_sso[df_sso.SPILL_ST_NAME.str.contains('ROOSEVELT AV') == True]

In [22]:
new_add = df_311.OBJECTDESC.str.split(', ', expand=True)

In [23]:
new_add.head(3)

,0,1,2,3,4
0,ELM CREEK and WURZBACH RD,None,None,None,None
1,1600 CENTENNIAL BV,San Antonio,78211,None,None
2,310 ROOSEVELT AV,San Antonio,78210,None,None


In [24]:
# rankings_pd.rename(columns = {'test':'TEST'}, inplace = True)

new_add = new_add.drop(columns=([3, 4]))
new_add = pd.DataFrame(new_add.rename(columns={0: 'street', 1: 'city', 2:'zip_code'}))

In [25]:
new_add.street

0          ELM CREEK and WURZBACH RD
1                1600  CENTENNIAL BV
2                  310  ROOSEVELT AV
3                  600  NOGALITOS ST
4                 6043  CASTLE QUEEN
                     ...            
571732              8911 SHADY WINDS
571733        6438 MAVERICK TRAIL DR
571734                  131 ADAMS ST
571735                 6039 TOPCROFT
571736     9318 JONES MALTSBERGER RD
Name: street, Length: 571737, dtype: object

In [26]:
def split_address(new_add, n):
    street_number = re.sub(r'\D', '', new_add.street[n])
    street_name = re.sub(r'\d', '', new_add.street[n])
    return street_number, street_name

In [28]:
street_number, street_name = split_address(new_add, 0)
street_number, street_name

('', 'ELM CREEK and WURZBACH RD')

In [32]:
new_add['stree_number', 'street_name'] 
new_add.apply(new_add)

ValueError: Length of values does not match length of index

In [ ]:
def convert_street_311(df):
    df[["street_address", 'city', 'zip_code', "something1", "something2"]] = df.OBJECTDESC.str.split(',', expand=True)
    return df